# Faults

How do models trained on only wind disturbances perform under full/partial motor losses? (Work in progress)

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from faults import FaultInjector
from environments import OctorotorEnvSelector
from utils import get_agent
from multirotor.helpers import DataLog

In [8]:
study, agent, params = get_agent("cardinal@vel@safesliding@dist") 
# study, agent, params = get_agent("cardinal@vel@bounding@dist") 

[I 2023-09-05 19:36:44,772] Using an existing study with name 'cardinal@vel@safesliding@dist' instead of creating a new one.


In [9]:
square_traj = np.array([[100,0,0], [100,100,0], [0,100,0], [0,0,0]])

In [10]:
oenv = OctorotorEnvSelector()
square_env = oenv.get_env("sliding", params, [(0,0),(0,0),(0,0)], square_traj)

In [11]:
# faulter = FaultInjector(square_env)
# square_env = faulter.inject_full_loss(0)

In [12]:
done = False
state = square_env.reset()
log = DataLog(square_env.base_env.vehicle, square_env.base_env.ctrl,
                  other_vars=('reward',))
vels = []
i = 0
while not done:
    if i == 20:
        faulter = FaultInjector(square_env)
        square_env = faulter.inject_partial_loss(0, 0.1)
        
    action = agent.predict(state, deterministic=True)[0]
    # action = [0,0,0]
    vels.append(action)
    state, reward, done, info = square_env.step(action)
    log.log(reward=reward)
    i += 1

vels = np.array(vels) * params['scaling_factor']/2
log.done_logging()

ValueError: Expected parameter loc (Tensor of shape (1, 3)) of distribution Normal(loc: torch.Size([1, 3]), scale: torch.Size([1, 3])) to satisfy the constraint Real(), but found invalid values:
tensor([[nan, nan, nan]], device='cuda:0')

In [ ]:
plt.plot(log.x, log.y)
# plt.xlim(-1,101)
# plt.ylim(-1,101)

In [ ]:
np.linspace(0, -2*np.pi, num=8, endpoint=False) + 0.375 * np.pi

In [ ]:
plt.plot(log.velocity)

In [ ]:
plt.plot(log.orientation)

In [ ]:
plt.plot(log.velocity[:,1], label='vel')
plt.plot(log.target.velocity[:,1], label='target vel')
plt.plot(vels[:,1], label='agent actions')
# plt.plot(log.target.position[:,0]) add in the change in waypoint
plt.legend()

In [ ]:
plt.plot(vels[:,1])

In [ ]:
plt.plot(log.z)

In [ ]:
info